# Automatic labelling of ground and buildings using data fusion

We have several options available to label ground and buildings:

* Using pre-processed AHN point clouds for ground and buildings.
* Using AHN GeoTIFF data for ground.
* Using BGT data for buidings (possibly combined with AHN).

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import logging

import src.fusion as fusion
from src.pipeline import Pipeline
import src.region_growing as growing
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.log_utils as log_utils
import src.utils.csv_utils as csv_utils
from src.labels import Labels

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

### Ground and building fuser using pre-processed AHN data

Prepare data following notebook [1. AHN preprocessing](1.%20AHN%20preprocessing.ipynb).

In [ ]:
# Data folder and file for the fusers.
ahn_data_folder = '../datasets/ahn/'
bgt_data_file = '../datasets/bgt/bgt_buildings_demo.csv'

# Create the reader for .npz data.
npz_reader = ahn_utils.NPZReader(ahn_data_folder)

# Ground fuser using pre-processed AHN data.
npz_ground_fuser = fusion.AHNFuser(Labels.GROUND, ahn_data_folder, ahn_reader=npz_reader,
                                   target='ground', epsilon=0.2)
# Building fuser using pre-processed AHN data.
npz_building_fuser = fusion.AHNFuser(Labels.BUILDING, ahn_data_folder, ahn_reader=npz_reader,
                                     target='building', epsilon=0.2)

### [Alternative] Ground fuser using GeoTIFF data

First, download the required GeoTIFF tile(s). E.g. for our demo point cloud, this is:
```sh
mkdir -p datasets/ahn
cd datasets/ahn/
wget https://download.pdok.nl/rws/ahn3/v1_0/05m_dtm/M_25DN2.ZIP
unzip M_25DN2.ZIP
rm M_25DN2.ZIP
```
Run the following cell to do this automatically.

In [ ]:
!mkdir -p ../datasets/ahn
!wget https://download.pdok.nl/rws/ahn3/v1_0/05m_dtm/M_25DN2.ZIP -P ../datasets/ahn/
!unzip ../datasets/ahn/M_25DN2.ZIP -d ../datasets/ahn/
!rm ../datasets/ahn/M_25DN2.ZIP

In [ ]:
# Data folder for the AHN fuser.
ahn_data_folder = '../datasets/ahn/'

# Create the reader for GeoTIFF data. See the class documentation for details on filling gaps and smoothening.
geotiff_reader = ahn_utils.GeoTIFFReader(ahn_data_folder,
                                         fill_gaps=True, max_gap_size=100,
                                         smoothen=True, smooth_thickness=2)

# Ground fuser using AHN GeoTIFF data.
geotiff_ground_fuser = fusion.AHNFuser(Labels.GROUND, ahn_data_folder, ahn_reader=geotiff_reader,
                                       target='ground', epsilon=0.2)

### [Alternative] Building fuser using BGT data
First, download and parse the required BGT data in notebook [1. Generate reference data](1.%20Generate%20reference%20data.ipynb). 

In [ ]:
# Building fuser using BGT building footprint data.
bgt_building_fuser = fusion.BGTBuildingFuser(Labels.BUILDING, bgt_file=bgt_data_file, building_offset=1)

### [Alternative] Building fuser using both BGT and AHN data
In this case, BGT will be used to define the <x, y> boundaries using the footprint polygon, and AHN data will be used to define the height <z> boundary.

In [ ]:
# Note: this only works with .npz (pre-processed) AHN data, since GeoTIFF does not contain building data.
bgt_ahn_building_fuser = fusion.BGTBuildingFuser(Labels.BUILDING, bgt_file=bgt_data_file, 
                                                 building_offset=0.25, ahn_reader=npz_reader)

### Region Growing

We use region growing to refine the buildings. This will make sure that protruding elements such as balconies are labelled correctly. For details and other options see notebook [Region growing](Region%20growing.ipynb).

In [ ]:
# We do this separately for the top and bottom since the best settings for each might differ.
building_top = {'bottom': 12., 'grid_size': 0.1, 'threshold': 0.5}
building_bottom = {'bottom': 0.5, 'top': 12., 'grid_size': 0.05, 'threshold': 0.5}

building_grower = growing.LayerLCC(Labels.BUILDING, npz_reader, 
                                   params=[building_top, building_bottom])

geotiff_building_grower = growing.LayerLCC(Labels.BUILDING, geotiff_reader, 
                                           params=[building_top, building_bottom])

## Process point clouds and save the results

The FusionPipeline can now be used to process point clouds. Labels will be written to the LAS file as a new _extended field_ called _label_.

In [ ]:
# Set-up full AHN pipeline.
process_sequence = (npz_ground_fuser, npz_building_fuser, building_grower)
pipeline = Pipeline(processors=process_sequence, caching=False)

In [ ]:
# ALTERNATIVE: Set-up AHN GeoTIFF + BGT pipeline.
process_sequence = (geotiff_ground_fuser, bgt_building_fuser, geotiff_building_grower)
pipeline = Pipeline(processors=process_sequence, caching=False)

In [ ]:
# BEST OPTION: Set-up AHN + BGT pipeline.
process_sequence = (npz_ground_fuser, bgt_ahn_building_fuser, building_grower)
pipeline = Pipeline(processors=process_sequence, caching=False)

### Process a single file

In [ ]:
# Select the file to process. The outfile can be set to 'None' to overwrite the file.
tilecode = '2386_9702'

in_file = '../datasets/pointcloud/filtered_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'

# Process the file.
pipeline.process_file(in_file, out_file=out_file)